These are our imports

In [2]:
#imports
import numpy as np
print("numpy imported")
import json
print("json imported")
import time
print("time imported")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print("tensorflow imported") 
from keras import Sequential
print("keras imported")
import config
channelID,serverID,targetID = config.getScope()
print("config imported")
import pickle
print("pickle imported")
import random
print("random imported")
import string
print("string imported")


numpy imported
json imported
time imported
tensorflow imported
keras imported
config imported
pickle imported
random imported
string imported


Run the section below to process your data

In [5]:
channelID,serverID,targetID = config.getScope()
daTestDoc = open("TestDoc","r",encoding="utf-8")
daMessageList = eval(daTestDoc.read())
with open('message.list','wb') as messageList_file:
    pickle.dump(daMessageList,messageList_file)
    print(":flushed:")

:flushed:


In [6]:
inputList,outputList = [],[]
with open("message.list",'rb') as daMessageList:
    channelID,serverID,targetID = config.getScope()
    daMessageList = pickle.load(daMessageList)

    for i in range(len(daMessageList)):
        message = daMessageList[i]
        if message["author"]["id"] == str(targetID):
            if type(message.get("referenced_message"))!=dict:
                if len(daMessageList) != i:
                    outputList.append(message["content"])
                    message = daMessageList[i+1]
                    inputList.append(message["content"])
            else:
                inputList.append(message["referenced_message"]["content"])
                outputList.append(message["content"])


    myIndice = random.randint(0,len(inputList))

    # for message in range(len(inputList)):
    # 	inputList[message] = removePunctuation(removeCaps(inputList[message]))

    with open('input.list','wb') as inputList_file:
        pickle.dump(inputList,inputList_file)
    with open('output.list','wb') as outputList_file:
        pickle.dump(outputList,outputList_file)
    print("Complete!")

Complete!


In [13]:
with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)
        
        removePunctuation = lambda x: ''.join(char for char in x if not char in string.punctuation)
        removeCaps = lambda x: x.lower()

        for message in range(len(inputList)):
            inputList[message] = removePunctuation(removeCaps(inputList[message]))
            outputList[message] = removePunctuation(removeCaps(outputList[message]))
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)
        print("Complete!")

Complete!


** This is a test to make sure the filters run correctly. **

In [14]:
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        for message in range(len(inputList)):
            for letter in ("abcdefghijklmnopqrstuvwxyz".upper()+string.punctuation):
                if letter in inputList[message]:
                    print(inputList[message])
                if letter in outputList[message]:
                    print(outputList[message])
print("!")

In [ ]:
#Create model
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)

In [ ]:


for msgNum in range(len(messageList)):         
    if messageList[msgNum]["author"]["id"] == "000000000000000000":
        print(messageList[msgNum])
        if "message_reference" in messageList[msgNum]:
            print("Reply")
            

In [ ]:
#View model info
model.summary()
